# Akka.Streams Integration with Akka.NET Actors
Behind the scenes, all Akka.Streams graphs are ultimately compiled via the `IMaterializer` into one or more Akka.NET actors. Thus there are a number of ways Akka.Streams can integrate with Akka.NET actors in real-world use.

## Integration with Actors in Flows
Using techniques such as the `Ask<TResult>` method in Akka.NET, we can incorporate actors directly inside built-in Akka.Streams flows without much code.

In [1]:
#r "nuget: Akka.Streams, 1.4.47"

using System.Linq;
using System.Collections.Immutable;
using Akka;
using Akka.Actor;
using Akka.Streams;
using Akka.Streams.Dsl;

ActorSystem actorSystem = ActorSystem.Create("StreamsExample");

IMaterializer materializer = actorSystem.Materializer();

// create a simple actor that performs some modulus
public class MyActor : ReceiveActor{
    public MyActor(){
        // returns a (i, i mod 2) ValueTuple
        Receive<int>(i => Sender.Tell((i, i % 2)));
    }
}

IActorRef myActorRef = actorSystem.ActorOf(Props.Create(() => new MyActor()), "myactor");

Installed Packages Akka.Streams, 1.4.47

In this instance, we've created a simple `ReceiveActor` that will perform a bit of modulus. Let's integrate it inside a simple graph and see how it works.

In [4]:
// create 10,000 integers
Source<int,NotUsed> source = Source.From(Enumerable.Range(0, 10));

// allow up to 10 parallel asks of our actor
var askFlow = Flow.Create<int>().SelectAsyncUnordered(10, async i => await myActorRef.Ask<(int value, int mod)>(i, TimeSpan.FromSeconds(3)));

// create a flow that does the following:
// int --> is even number? --> keep only even numbers --> count total even numbers
IAsyncEnumerable<int> graph = source.Via(askFlow).Where(d => d.mod == 0).Select(x => x.value).RunAsAsyncEnumerable(materializer);

await foreach(var i in graph){
    Console.WriteLine(i);
}

0
2
4
6
8


In this case, we're using the `SelectAsync` stage, which allows for up to N `Task<T>`s to run in parallel before back-pressuring the stage above, to `Ask<T>` our `ReceiveActor` we created earlier. The actor will do all of the processing and upon replying to the sender will complete the `Task<T>` that was being `await`-ed upon inside the `SelectAsync` statement.